In [ ]:
# Loading dataset from Google Drive. Upload the given dataset to Google Drive first

from google.colab import drive
drive.mount('/content/gdrive')

!unzip /content/gdrive/MyDrive/Custom_Dataset.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/Custom_Dataset/Test/Peace/peace (194).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (195).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (196).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (197).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (198).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (199).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (2).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (20).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (200).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (201).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (202).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (203).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (204).jpg  
  inflating: /content/Custom_Dataset/Test/Peace/peace (205).jpg  
  inflating: /content/Custom

In [ ]:
# Importing required libraries and invoking GPU

import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', tf.keras.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

train_dir = '/content/Custom_Dataset/Train'
test_dir = '/content/Custom_Dataset/Test'

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Using:
	• TensorFlow version: 2.12.0
	• tf.keras version: 2.12.0
	• GPU device not found. Running on CPU


In [ ]:
# Loading images from dataset

BATCH_SIZE = 32
IMG_SHAPE  = 16

image_gen_train = ImageDataGenerator(rescale=1./255, shear_range=20, zoom_range=0.2)
image_gen_test = ImageDataGenerator(rescale=1./255)

train_data_gen = image_gen_train.flow_from_directory(directory=train_dir,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
                                          target_size=(IMG_SHAPE,IMG_SHAPE),
                                          color_mode='grayscale',
                                          class_mode='categorical')

test_data_gen = image_gen_test.flow_from_directory(directory=test_dir,
                                                 batch_size=BATCH_SIZE,
                                                 target_size=(IMG_SHAPE,IMG_SHAPE),
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

Found 4800 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.


In [ ]:
# Defining neural network architecture

layer_neurons = [16,16,16,16]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = (IMG_SHAPE, IMG_SHAPE, 1)))

for neurons in layer_neurons:
    model.add(tf.keras.layers.Dense(neurons, activation='relu'))
            
model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 4)                 68        
                                                                 
Total params: 9,364
Trainable params: 9,364
Non-trainabl

In [ ]:
# Compiling the model using Adam optimizer, caterogiral crossentropy loss function and accuracy as performance metric

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training the model using the dataset for 10 epochs

history = model.fit(train_data_gen,
                    epochs=10,
                    validation_data=test_data_gen)

Epoch 1/10
150/150 [==============================] - 9s 37ms/step - loss: 1.0547 - accuracy: 0.5529 - val_loss: 0.6248 - val_accuracy: 0.7925
Epoch 2/10
150/150 [==============================] - 5s 32ms/step - loss: 0.5280 - accuracy: 0.8123 - val_loss: 0.4535 - val_accuracy: 0.8342
Epoch 3/10
150/150 [==============================] - 6s 38ms/step - loss: 0.3740 - accuracy: 0.8719 - val_loss: 0.2756 - val_accuracy: 0.9133
Epoch 4/10
150/150 [==============================] - 5s 36ms/step - loss: 0.2735 - accuracy: 0.9104 - val_loss: 0.2102 - val_accuracy: 0.9367
Epoch 5/10
150/150 [==============================] - 5s 31ms/step - loss: 0.2020 - accuracy: 0.9346 - val_loss: 0.1751 - val_accuracy: 0.9325
Epoch 6/10
150/150 [==============================] - 6s 43ms/step - loss: 0.1508 - accuracy: 0.9538 - val_loss: 0.1611 - val_accuracy: 0.9475
Epoch 7/10
150/150 [==============================] - 5s 31ms/step - loss: 0.1339 - accuracy: 0.9550 - val_loss: 0.1598 - val_accuracy: 0.9367

In [ ]:
# Save model (optional)
model.save('hgr_model')

# Extract weights and biases from the model and save them as csv files

w1 = np.array(model.layers[1].weights[0])
b1 = np.array(model.layers[1].weights[1])

w2 = np.array(model.layers[2].weights[0])
b2 = np.array(model.layers[2].weights[1])

w3 = np.array(model.layers[3].weights[0])
b3 = np.array(model.layers[3].weights[1])

w4 = np.array(model.layers[4].weights[0])
b4 = np.array(model.layers[4].weights[1])

w5 = np.array(model.layers[5].weights[0])
b5 = np.array(model.layers[5].weights[1])

w1 = pd.DataFrame(w1)
w2 = pd.DataFrame(w2)
w3 = pd.DataFrame(w3)
w4 = pd.DataFrame(w4)
w5 = pd.DataFrame(w5)
b1 = pd.DataFrame(b1)
b2 = pd.DataFrame(b2)
b3 = pd.DataFrame(b3)
b4 = pd.DataFrame(b4)
b5 = pd.DataFrame(b5)

w1.to_csv('w1.csv')
w2.to_csv('w2.csv')
w3.to_csv('w3.csv')
w4.to_csv('w4.csv')
w5.to_csv('w5.csv')
b1.to_csv('b1.csv')
b2.to_csv('b2.csv')
b3.to_csv('b3.csv')
b4.to_csv('b4.csv')
b5.to_csv('b5.csv')